Scrape name list of MIT EECS faculty members

In [ ]:
import urllib2
from bs4 import BeautifulSoup

page = urllib2.urlopen("https://www.eecs.mit.edu/people/faculty-advisors")
soup = BeautifulSoup(page,'html.parser')
faculty = []

faculty_rows = soup.find_all('div',attrs={'class':'views-field views-field-title'})
for i in range(len(faculty_rows)):
    faculty.append(faculty_rows[i].getText())
    



Query for papers in arXiv for each faculty member and output each abstract in a separate text file. All abstract text files stored in "paper_abstracts" folder.

In [ ]:
for faculty_member in faculty:
    first_name = faculty_member.split()[0]
    last_name = faculty_member.split()[-1]
    if last_name == "Jr.":
        last_name = faculty_member.split()[-2] 
    search_url = "https://" + "arxiv.org/find/all/1/au:+" + last_name.encode('utf-8') + "_" + first_name.encode('utf-8') + "/0/1/0/all/0/1"
    req = urllib2.Request(search_url, headers={'User-Agent' : 'Magic Browser'})
    author_page = urllib2.urlopen(req)
    html = BeautifulSoup(author_page, 'html.parser')
    paper_rows = html.find_all('a', attrs = {'title':'Abstract'})
    paper_number = 1
    for paper in paper_rows:
        identifier = paper.getText()[6:]
        abstract_url = "https://arxiv.org/abs/" + identifier
        abstract_req = urllib2.Request(abstract_url, headers={'User-Agent' : 'Magic Browser'})
        abstract_page = urllib2.urlopen(abstract_req)
        abstract_html = BeautifulSoup(abstract_page, 'html.parser')
        description = abstract_html.find('blockquote', attrs = {'class':'abstract mathjax'}).getText()
        title = abstract_html.find('h1', attrs = {'class': 'title mathjax'}).getText()
        with open(last_name.encode('utf-8')+"_"+first_name.encode('utf-8')+"_"+str(paper_number) +".txt", 'w') as text_file:
            text_file.write(title.encode('utf-8') + description.encode('utf-8'))
        paper_number = paper_number + 1
    

Create list of documents and store in "all_docs.txt"

In [ ]:
import glob
filenames = glob.glob('paper_abstracts/*.txt')
all_docs = open('all_docs.txt','w')
for file in filenames:
    print>>all_docs, file

Create dictionary of words, clean, and store in "dictionary_clean.csv"

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import csv
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


##EXTRACT ALL WORDS
all_words = []
list_of_docs = "all_docs.txt"
tokenizer = RegexpTokenizer(r'\w+')


with open(list_of_docs, 'r') as f:
	docs = f.readlines()

for doc in docs:
	doc_entry = doc.strip()
	f = open(doc_entry,'r')
	doc_text = f.read()
	raw = doc_text.lower()
	tokens = tokenizer.tokenize(raw)
	for token in tokens:
		all_words.append(token)

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)



##REMOVE STOPWORDS/PUNCTUATION
dictionary_all = []
for i in all_words:
	if i not in punctuation and i not in stopwords:
		dictionary_all.append(i)


##LEMMATIZE TO REDUCE RELATED FORMS OF A WORD TO A COMMON BASE AND DEDUPE
lemmatize = WordNetLemmatizer()
dictionary_clean = []
for i in dictionary_all:
	lemmatized_word = lemmatize.lemmatize(i.decode('utf8','ignore'))
	if lemmatized_word not in dictionary_clean:
		dictionary_clean.append(lemmatized_word.encode('utf8'))


##READ TO CSV
csvfile = "dictionary_clean.csv"
with open(csvfile, 'w') as output:
	writer = csv.writer(output, lineterminator='\n')
	for val in dictionary_clean:
		writer.writerow([val])
